In [13]:
import pandas as pd

xls = pd.ExcelFile("Scats Data October 2006.xls")

# Show all sheet names in the Excel file
print(xls.sheet_names)


['Notes', 'Data', 'Summary Of Data']


In [23]:
print(df_clean.columns.tolist())


['Start Time', datetime.time(0, 0), datetime.time(0, 15), datetime.time(0, 30), datetime.time(0, 45), datetime.time(1, 0), datetime.time(1, 15), datetime.time(1, 30), datetime.time(1, 45), datetime.time(2, 0), datetime.time(2, 15), datetime.time(2, 30), datetime.time(2, 45), datetime.time(3, 0), datetime.time(3, 15), datetime.time(3, 30), datetime.time(3, 45), datetime.time(4, 0), datetime.time(4, 15), datetime.time(4, 30), datetime.time(4, 45), datetime.time(5, 0), datetime.time(5, 15), datetime.time(5, 30), datetime.time(5, 45), datetime.time(6, 0), datetime.time(6, 15), datetime.time(6, 30), datetime.time(6, 45), datetime.time(7, 0), datetime.time(7, 15), datetime.time(7, 30), datetime.time(7, 45), datetime.time(8, 0), datetime.time(8, 15), datetime.time(8, 30), datetime.time(8, 45), datetime.time(9, 0), datetime.time(9, 15), datetime.time(9, 30), datetime.time(9, 45), datetime.time(10, 0), datetime.time(10, 15), datetime.time(10, 30), datetime.time(10, 45), datetime.time(11, 0), da

In [29]:
import pandas as pd
import datetime

# Step 1: Drop all-empty rows and the first 9 metadata columns
df_clean = df.dropna(how='all')
df_clean = df_clean.drop(columns=df_clean.columns[:9])

# ✅ Step 2: Remove rows where 'Start Time' is not a proper datetime (like "Date" text)
df_clean = df_clean[~df_clean['Start Time'].astype(str).str.contains("Date", case=False)]

# ✅ Step 3: Convert 'Start Time' to datetime safely
df_clean['Start Time'] = pd.to_datetime(df_clean['Start Time'], errors='coerce')
df_clean = df_clean.dropna(subset=['Start Time'])  # Remove any failed conversions

# Step 4: Melt into long format
df_melted = df_clean.melt(id_vars=["Start Time"], var_name="Time", value_name="Volume")

# Step 5: Combine date + time
df_melted['Datetime'] = df_melted.apply(
    lambda row: datetime.datetime.combine(row['Start Time'].date(), row['Time']), axis=1
)

# Step 6: Extract ML features
df_melted['Hour'] = df_melted['Datetime'].dt.hour
df_melted['DayOfWeek'] = df_melted['Datetime'].dt.dayofweek
df_melted = df_melted.dropna(subset=['Volume'])

# ✅ Final preview
print(df_melted.head())


           Start Time      Time Volume   Datetime  Hour  DayOfWeek
0 2006-10-01 00:15:00  00:00:00     86 2006-10-01     0          6
1 2006-10-02 00:15:00  00:00:00     32 2006-10-02     0          0
2 2006-10-03 00:15:00  00:00:00     26 2006-10-03     0          1
3 2006-10-04 00:15:00  00:00:00     32 2006-10-04     0          2
4 2006-10-05 00:15:00  00:00:00     40 2006-10-05     0          3


In [33]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Step 1: Select input features and target
X = df_melted[['Hour', 'DayOfWeek']]
y = df_melted['Volume']

# Step 2: Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Step 3: Train the Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Step 4: Predict and evaluate
predictions = model.predict(X_test)

mae = mean_absolute_error(y_test, predictions)
rmse = np.sqrt(mean_squared_error(y_test, predictions))

print("✅ MAE:", mae)
print("✅ RMSE:", rmse)


✅ MAE: 164.43632918885118
✅ RMSE: 172.8960457333665


In [37]:
# Converts volume (flow) → speed using the quadratic formula
def flow_to_speed(flow):
    a = -1.4648375
    b = 93.75
    c = -flow
    discriminant = b**2 - 4*a*c
    if discriminant < 0:
        return 5  # fallback speed
    sqrt_d = np.sqrt(discriminant)
    speed1 = (-b + sqrt_d) / (2*a)
    speed2 = (-b - sqrt_d) / (2*a)
    return max(min(speed1, speed2), 1)  # choose under-saturated speed

# Converts speed → travel time in seconds
def speed_to_travel_time(speed_kmh, distance_km=1):
    travel_time_sec = (distance_km / speed_kmh) * 3600 + 30
    return travel_time_sec


In [39]:
# Try on one prediction
predicted_flow = predictions[0]
speed = flow_to_speed(predicted_flow)
travel_time = speed_to_travel_time(speed)

print(f"🚦 Predicted flow: {predicted_flow:.2f}")
print(f"🚗 Estimated speed: {speed:.2f} km/h")
print(f"🕒 Travel time: {travel_time:.2f} seconds")


🚦 Predicted flow: 217.33
🚗 Estimated speed: 2.41 km/h
🕒 Travel time: 1524.52 seconds
